In [1]:
import tensorflow as tf
import os
from langchain_google_genai import GoogleGenerativeAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain



c:\Users\ULTRON\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=Your-API-KEY)
print(
    llm.invoke(
        "What are some of the pros and cons of Python as a programming language?"
    )
)

**Pros of Python**

* **Easy to learn:** Python is a very easy language to learn, even for beginners. The syntax is simple and straightforward, and there are a lot of resources available to help you get started.
* **Versatile:** Python can be used for a wide variety of tasks, including web development, data science, and machine learning. It's also a good choice for beginners because it can be used for so many different things.
* **High-level:** Python is a high-level language, which means that it's closer to human language than some other programming languages. This makes it easier to read and write code, and it also makes it more portable.
* **Community:** Python has a large and active community, which means that there are a lot of resources available to help you learn and use the language. There are also a lot of people who are willing to help you if you get stuck.

**Cons of Python**

* **Slow:** Python is not as fast as some other programming languages, such as C++ or Java. This is

In [3]:
db_user = 'root'
db_password = 'root'
db_host = 'localhost'
db_name = 'moviesdb'

db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)

print(db.table_info)


CREATE TABLE actors (
	actor_id INTEGER UNSIGNED NOT NULL AUTO_INCREMENT, 
	name VARCHAR(45) NOT NULL, 
	birth_year YEAR, 
	PRIMARY KEY (actor_id)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from actors table:
actor_id	name	birth_year
50	Yash	1986
51	Sanjay Dutt	1959
52	Benedict Cumberbatch	1976
*/


CREATE TABLE financials (
	movie_id INTEGER UNSIGNED NOT NULL, 
	budget DECIMAL(10, 2), 
	revenue DECIMAL(10, 2), 
	unit ENUM('Units','Thousands','Millions','Billions'), 
	currency CHAR(3), 
	PRIMARY KEY (movie_id)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows from financials table:
movie_id	budget	revenue	unit	currency
101	1.00	12.50	Billions	INR
102	200.00	954.80	Millions	USD
103	165.00	644.80	Millions	USD
*/


CREATE TABLE languages (
	language_id TINYINT UNSIGNED NOT NULL AUTO_INCREMENT, 
	name VARCHAR(45) NOT NULL, 
	PRIMARY KEY (language_id)
)DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci

/*
3 rows fr

In [4]:
db_chain = SQLDatabaseChain.from_llm(llm, db,verbose=True)

# Contoh mudah
qns1 = db_chain.run("How many movies do yo have?")

c:\Users\ULTRON\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
How many movies do yo have?
SQLQuery:SELECT count(*) FROM movies
SQLResult: [(39,)]
Answer:39
> Finished chain.


In [5]:
# contoh sulit
qns2 = db_chain.run("How many movies is using telugu language?")



> Entering new SQLDatabaseChain chain...
How many movies is using telugu language?
SQLQuery:SELECT count(*) FROM movies WHERE language_id=3
SQLResult: [(1,)]
Answer:1
> Finished chain.


In [6]:
# SELECT COUNT(*) FROM movies WHERE language_id IN (Select language_id FROM languages WHERE name="TELUGU")

In [7]:
# Contoh sult+
qns3 = db_chain.run("Which Hollywood film has the biggest profit ever?")

# Harus mikir revenue - budget, unit, dan currency



> Entering new SQLDatabaseChain chain...
Which Hollywood film has the biggest profit ever?
SQLQuery:SELECT title FROM movies JOIN financials ON movies.movie_id = financials.movie_id WHERE movies.industry = "Hollywood" ORDER BY revenue - budget DESC LIMIT 1
SQLResult: [('Avatar',)]
Answer:Avatar
> Finished chain.


In [8]:
# SELECT mt.movie_id, mt.title
# FROM (
#     SELECT movie_id, 
#         (CASE 
#             WHEN unit = 'Billions' AND currency = 'INR' THEN (revenue * 1000 * 0.012) - (budget * 1000 * 0.012)
#             WHEN unit = 'Millions' AND currency = 'INR' THEN (revenue * 0.012) - (budget * 0.012)
#             WHEN unit = 'Billions' AND currency = 'USD' THEN (revenue * 1000) - (budget * 1000)
#             ELSE revenue - budget
#         END) AS profit_in_usd_millions
#     FROM financials
#     ORDER BY profit_in_usd_millions DESC
#     LIMIT 1
# ) AS max_profit
# JOIN movies mt ON max_profit.movie_id = mt.movie_id;

In [9]:
qns4 = db_chain.run("Please suggest me good holywood movie")



> Entering new SQLDatabaseChain chain...
Please suggest me good holywood movie
SQLQuery:SELECT title FROM movies WHERE industry = 'Hollywood' AND imdb_rating > 7.0 ORDER BY release_year DESC LIMIT 5
SQLResult: [('Parasite',), ('Avengers: Endgame',), ('Avengers: Infinity War',), ('Thor: Ragnarok ',), ('Captain America: The Winter Soldier',)]
Answer:Parasite, Avengers: Endgame, Avengers: Infinity War, Thor: Ragnarok , Captain America: The Winter Soldier
> Finished chain.


Few Shot Learning

In [10]:
qnsx = db_chain.run("How many movies is using telugu language?")
qnsy = db_chain.run("Which Hollywood film has the biggest profit ever?")
qnsa = db_chain.run("Please suggest me good holywood movie")




> Entering new SQLDatabaseChain chain...
How many movies is using telugu language?
SQLQuery:SELECT count(*) FROM movies JOIN languages ON movies.language_id = languages.language_id WHERE languages.name = 'Telugu'
SQLResult: [(3,)]
Answer:3
> Finished chain.


> Entering new SQLDatabaseChain chain...
Which Hollywood film has the biggest profit ever?
SQLQuery:SELECT title FROM movies JOIN financials ON movies.movie_id = financials.movie_id WHERE movies.industry = 'Hollywood' ORDER BY financials.revenue - financials.budget DESC LIMIT 1
SQLResult: [('Avatar',)]
Answer:Avatar
> Finished chain.


> Entering new SQLDatabaseChain chain...
Please suggest me good holywood movie
SQLQuery:SELECT title FROM movies WHERE industry='Hollywood' AND imdb_rating>=7.0 ORDER BY release_year DESC LIMIT 5
SQLResult: [('Doctor Strange in the Multiverse of Madness',), ('Avengers: Endgame',), ('Parasite',), ('Avengers: Infinity War',), ('Thor: Ragnarok ',)]
Answer:Doctor Strange in the Multiverse of Madness, 

In [11]:
few_shots = [
    {
        'Question': "How many movies is using telugu language?",
        'SQLQuery': "SELECT COUNT(*) FROM movies WHERE language_id IN (SELECT language_id FROM languages WHERE name='Telugu')",
        'SQLResult': "Result of the SQL query",
        'Answer': "3"
    },
    {
        'Question': "Which Hollywood film has the biggest profit ever?",
        'SQLQuery': "SELECT mt.title FROM (SELECT movie_id, (CASE WHEN unit = 'Billions' AND currency = 'INR' THEN (revenue * 1000 * 0.012) - (budget * 1000 * 0.012) WHEN unit = 'Millions' AND currency = 'INR' THEN (revenue * 0.012) - (budget * 0.012) WHEN unit = 'Billions' AND currency = 'USD' THEN (revenue * 1000) - (budget * 1000) ELSE revenue - budget END) AS profit_in_usd_millions FROM financials ORDER BY profit_in_usd_millions DESC LIMIT 1) AS max_profit JOIN movies mt ON max_profit.movie_id = mt.movie_id;",
        'SQLResult': "Result of the SQL query",
        'Answer': "Avatar"
    },
    {
        'Question': "Please suggest me good holywood movie",
        'SQLQuery': "SELECT title FROM movies WHERE industry = 'Hollywood' AND imdb_rating > 7 ORDER BY imdb_rating DESC LIMIT 1",
        'SQLResult': "Result of the SQL query",
        'Answer': "The Shawshank Redemption"
    }
]

In [12]:
from langchain.embeddings import HuggingFaceEmbeddings

# HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
e = embeddings.embed_query("How many movies do yo have?")

c:\Users\ULTRON\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\ULTRON\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
len(e)

384

In [14]:
to_vectorize = [" ".join(example.values()) for example in few_shots]
to_vectorize

["How many movies is using telugu language? SELECT COUNT(*) FROM movies WHERE language_id IN (SELECT language_id FROM languages WHERE name='Telugu') Result of the SQL query 3",
 "Which Hollywood film has the biggest profit ever? SELECT mt.title FROM (SELECT movie_id, (CASE WHEN unit = 'Billions' AND currency = 'INR' THEN (revenue * 1000 * 0.012) - (budget * 1000 * 0.012) WHEN unit = 'Millions' AND currency = 'INR' THEN (revenue * 0.012) - (budget * 0.012) WHEN unit = 'Billions' AND currency = 'USD' THEN (revenue * 1000) - (budget * 1000) ELSE revenue - budget END) AS profit_in_usd_millions FROM financials ORDER BY profit_in_usd_millions DESC LIMIT 1) AS max_profit JOIN movies mt ON max_profit.movie_id = mt.movie_id; Result of the SQL query Avatar",
 "Please suggest me good holywood movie SELECT title FROM movies WHERE industry = 'Hollywood' AND imdb_rating > 7 ORDER BY imdb_rating DESC LIMIT 1 Result of the SQL query The Shawshank Redemption"]

In [15]:
from langchain.vectorstores import Chroma

vector_store = Chroma.from_texts(to_vectorize, embedding=embeddings, metadatas=few_shots)

In [16]:
from langchain.prompts import SemanticSimilarityExampleSelector

example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vector_store,
    k=2
)

example_selector.select_examples({'Question' : "How many movies is using English language?"})

[{'Answer': '3',
  'Question': 'How many movies is using telugu language?',
  'SQLQuery': "SELECT COUNT(*) FROM movies WHERE language_id IN (SELECT language_id FROM languages WHERE name='Telugu')",
  'SQLResult': 'Result of the SQL query'},
 {'Answer': 'The Shawshank Redemption',
  'Question': 'Please suggest me good holywood movie',
  'SQLQuery': "SELECT title FROM movies WHERE industry = 'Hollywood' AND imdb_rating > 7 ORDER BY imdb_rating DESC LIMIT 1",
  'SQLResult': 'Result of the SQL query'}]

In [17]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

In [18]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [19]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [20]:
from langchain.prompts.prompt import PromptTemplate

example_prompt = PromptTemplate(
    input_variables=['Question', 'SQLQuery', 'SQLResult', 'Answer',],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}"
)

In [21]:
from langchain.prompts import FewShotPromptTemplate

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=_mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"],
)

In [22]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [23]:
new_chain("How many movies is using telugu language?")



> Entering new SQLDatabaseChain chain...
How many movies is using telugu language?
SQLQuery:

c:\Users\ULTRON\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


SELECT COUNT(*) FROM movies WHERE language_id IN (SELECT language_id FROM languages WHERE name='Telugu')
SQLResult: [(3,)]
Answer:3
> Finished chain.


{'query': 'How many movies is using telugu language?', 'result': '3'}

In [24]:
new_chain("Which Hollywood film has the biggest profit ever?")



> Entering new SQLDatabaseChain chain...
Which Hollywood film has the biggest profit ever?
SQLQuery:SELECT mt.title FROM (SELECT movie_id, (CASE WHEN unit = 'Billions' AND currency = 'INR' THEN (revenue * 1000 * 0.012) - (budget * 1000 * 0.012) WHEN unit = 'Millions' AND currency = 'INR' THEN (revenue * 0.012) - (budget * 0.012) WHEN unit = 'Billions' AND currency = 'USD' THEN (revenue * 1000) - (budget * 1000) ELSE revenue - budget END) AS profit_in_usd_millions FROM financials ORDER BY profit_in_usd_millions DESC LIMIT 1) AS max_profit JOIN movies mt ON max_profit.movie_id = mt.movie_id;
SQLResult: [('Avatar',)]
Answer:Avatar
> Finished chain.


{'query': 'Which Hollywood film has the biggest profit ever?',
 'result': 'Avatar'}